Data source:
- GitHub mirror repo containing SupplyChainLogisticsProblems.xlsx:
  https://github.com/jaredbach/LogisticsDataset  (downloaded as zip)


In [3]:
from __future__ import annotations

import io
import os
import re
import sys
import zipfile
from pathlib import Path
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import cvxpy as cp
import requests


In [13]:
WORKDIR = Path("./supply_chain_lp_work")
RAW_DIR = WORKDIR / "raw"
EXTRACT_DIR = RAW_DIR / "repo_extract"
OUT_DIR = WORKDIR / "out"

WORKDIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
# For big instances, you can restrict candidate plants per order to keep the LP smaller:
MAX_PLANTS_PER_ORDER = 10  # set to None for no restriction

# If costs are per weight unit, you might want to scale weight; keep as-is by default
WEIGHT_SCALE = 1.0



In [19]:
# -----------------------------
# Utilities: download & extract
# -----------------------------
def download_zip(url: str, dest_path: Path) -> None:
    dest_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"Downloading: {url}")
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    dest_path.write_bytes(r.content)
    print(f"Saved zip to: {dest_path.resolve()}")


def extract_zip(zip_path: Path, dest_dir: Path) -> None:
    dest_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(dest_dir)
    print(f"Extracted zip to: {dest_dir.resolve()}")


def find_excel_file(root: Path) -> Path:
    # Look for the known workbook name OR any xlsx
    candidates = list(root.rglob("SupplyChainLogisticsProblems.xlsx"))
    if candidates:
        return candidates[0]
    candidates = list(root.rglob("*.xlsx"))
    if not candidates:
        raise FileNotFoundError(f"No .xlsx found under {root}")
    return candidates[0]

# -----------------------------
# Cleaning helpers
# -----------------------------
def canon(s: str) -> str:
    """Canonical column name: lowercase, strip, replace non-alnum with underscore."""
    s = s.strip().lower()
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s


def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [canon(c) for c in df.columns]
    # Normalize common missing value tokens
    df = df.replace({"": np.nan, "NA": np.nan, "N/A": np.nan, "null": np.nan})
    return df


def load_excel_sheets(xlsx_path: Path) -> Dict[str, pd.DataFrame]:
    print(f"Loading workbook: {xlsx_path.resolve()}")
    xls = pd.ExcelFile(xlsx_path)
    sheets = {}
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        df = clean_df(df)
        sheets[canon(sheet_name)] = df
    print(f"Loaded {len(sheets)} sheets: {list(sheets.keys())}")
    return sheets


def pick_sheet(sheets: Dict[str, pd.DataFrame], preferred: List[str]) -> Optional[pd.DataFrame]:
    for key in preferred:
        if key in sheets:
            return sheets[key]
    return None


def require_columns(df: pd.DataFrame, required: List[str], context: str) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(
            f"[{context}] Missing required columns: {missing}\n"
            f"Available columns: {list(df.columns)}"
        )



In [15]:
zip_path = RAW_DIR / "LogisticsDataset_main.zip"
GITHUB_ZIP_URL = "https://github.com/jaredbach/LogisticsDataset/archive/refs/heads/main.zip"

In [ ]:
if not zip_path.exists():
    download_zip(GITHUB_ZIP_URL, zip_path)
else:
    print(f"Zip already exists: {zip_path.resolve()}")

if EXTRACT_DIR.exists():
        # keep it simple: don't re-extract unless user deletes folder
    print(f"Extract dir exists: {EXTRACT_DIR.resolve()}")
else:
    extract_zip(zip_path, EXTRACT_DIR)



Zip already exists: C:\Users\salmank\Documents\convex_optimization\docs\convex\tutorials\supply_chain_lp_work\raw\LogisticsDataset_main.zip
Extracted zip to: C:\Users\salmank\Documents\convex_optimization\docs\convex\tutorials\supply_chain_lp_work\raw\repo_extract


In [23]:
xlsx_path = find_excel_file(EXTRACT_DIR)
sheets = load_excel_sheets(xlsx_path)

Loading workbook: C:\Users\salmank\Documents\convex_optimization\docs\convex\tutorials\supply_chain_lp_work\raw\repo_extract\LogisticsDataset-main\SupplyChainLogisticsProblems.xlsx
Loaded 7 sheets: ['orderlist', 'freightrates', 'whcosts', 'whcapacities', 'productsperplant', 'vmicustomers', 'plantports']


In [32]:
sheets['orderlist']

,order_id,order_date,orig_port,carrier,tpt_day_count,service_level,ship_ahead_day_count,ship_late_day_count,customer,product_id,plant_code,dest_port,unit_quant,weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.300000
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.940000
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.200000
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.160000
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9210,1.447305e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9211,1.447319e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9212,1.447322e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,245,0.294265
9213,1.447145e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683430,PLANT03,PORT09,278,2.480000


In [ ]:
## Demand measure
orders  = sheets['orderlist'][['order_id','weight']]
orders.columns = ["order_id", "demand"]
orders = orders[orders["demand"] > 0].copy()
orders["order_id"] = orders["order_id"].astype(str)
orders.head()

,order_id,demand
0,1447296446.7,14.30
1,1447158014.7,87.94
2,1447138898.7,61.20
3,1447363527.7,16.16
4,1447363980.7,52.34


In [ ]:
# Plants and products
plants = sheets['productsperplant']
plants.head()


,plant_code,product_id
0,PLANT15,1698815
1,PLANT17,1664419
2,PLANT17,1664426
3,PLANT17,1672826
4,PLANT17,1674916


In [ ]:
# Plant capacities
cap = sheets['whcapacities']
cap.columns = ["plant_id", "supply_cap"]
cap["plant_id"] = cap["plant_id"].astype(str)
cap["supply_cap"] = pd.to_numeric(cap["supply_cap"], errors="coerce")
cap = cap.dropna(subset=["supply_cap"])
#plants = plants.merge(cap, on="plant_id", how="left")
cap

,plant_id,supply_cap
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138
5,PLANT01,1070
6,PLANT06,49
7,PLANT10,118
8,PLANT07,265
9,PLANT14,549


In [ ]:
sheets.keys()

dict_keys(['orderlist', 'freightrates', 'whcosts', 'whcapacities', 'productsperplant', 'vmicustomers', 'plantports'])